In [1]:
%load_ext Cython

In [3]:
import sys
import pandas as pd
import numpy as np
sys.path.append('/home/pablo-ubuntu/Dropbox/PI/PI2/code')
inputs = pd.read_csv('./test_cases/inputs50.csv', index_col=0)

import model as m1


import model_vJCR as m2


In [107]:
%%cython -a


cimport cython

import numpy as np
cimport numpy as np

@cython.boundscheck(False)
@cython.wraparound(False)
def cy_medal_organizer(np.double_t[:] d,np.double_t[:] c, np.int64_t [:] l, int n):
    cdef int ind = 0
    l[0] = 0
    cdef int i
    for i in range(1, n):
        if d[i] == d[i-1] and c[i] == c[i-1]:
            l[i] = l[i-1]
        elif d[i] == d[ind]:
            l[i] = l[i-1] + 1
        elif c[i] < c[ind]:
            l[i] = 0
            ind = i
        else:
            l[i] = 1
            for j in range(1, i):
                if d[i] > d[j] and c[i] > c[j] and l[j] >= l[i]:
                    l[i] = l[j] + 1

In [110]:

def organize_medal_table(table, medal_max=ValueError):
    """Organize medal table pensado por Juan Carlos"""
    org_criteria = ['Unb', 'Tsp']
    table.sort_values(org_criteria, inplace=True)
    n = len(table)
    # d = list(table['Unb'])
    # c = list(table['Tsp'])
    d = table.values[:, 0]
    c = table.values[:, 1]
    l = np.repeat(-1, len(c))
    cy_medal_organizer(d, c, l, n)
    table['Pod_lev'] = l
    return table, olimpic_results(table)

def olimpic_results(table):
    metals = ['oro', 'plata', 'bronce']
    cut_table = table[['Sol', 'Pod_lev']]
    n_sols ,high_pod_lev = cut_table.max() + 1
    medal_table = np.zeros([n_sols, high_pod_lev], dtype=int)
    for row in cut_table.itertuples():
        __, iSol, lev_pod = row
        medal_table[iSol, lev_pod] += 1
    medal_table = pd.DataFrame(medal_table)
    cols = list(medal_table.columns)
    medal_table.sort_values(by=cols, inplace=True, ascending=False)
    medal_table.index.name = 'Soluciones'
    medal_table.columns.name = 'Medallas'
    return medal_table

## Comparación algoritmos con cython

In [111]:
%timeit m1.organize_medal_table(inputs)
%timeit m2.organize_medal_table(inputs)
%timeit organize_medal_table(inputs)

6.83 ms ± 28.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
6.75 ms ± 54.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
6.04 ms ± 54.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [116]:
6.83/6 -1

0.13833333333333342

In [113]:
organize_medal_table(inputs)

(       Unb           Tsp  Sol  Pod_lev
 148   62.0  15841.030370    2        0
 149   62.0  16370.034151    2        1
 45    68.0  13592.395325    0        0
 46    68.0  13592.395325    0        0
 47    68.0  13592.395325    0        0
 ..     ...           ...  ...      ...
 7    198.0    544.022977    0        1
 104  198.0    568.636967    2        2
 105  198.0    568.636967    2        2
 52   198.0    786.018448    1        3
 53   198.0    786.018448    1        3
 
 [250 rows x 4 columns], Medallas     0   1   2   3   4  5  6  7  8
 Soluciones                                
 3           29   7   8   1   5  0  0  0  0
 0           10  15   7   3   7  4  4  0  0
 4            7   8  19   9   4  3  0  0  0
 2            6   5   9  10  10  1  3  4  2
 1            5   2   9  19  10  2  0  3  0)